# Messing with HRRR Grib data on AWS

In [ ]:
import fsspec
import xarray as xr
import hvplot.xarray
import metpy
import numpy as np

In [ ]:
fs = fsspec.filesystem('s3', anon=True)

In [ ]:
# url = 'noaa-hrrr-pds'   # forecast data

In [ ]:
url = 's3://noaa-hrrr-bdp-pds/hrrr.20190101/conus/hrrr.t12z.wrfsfcf01.grib2'

In [ ]:
%%time
file = fsspec.open(url)

tmp_file = fsspec.open_local(f'simplecache::{url}', 
                              s3=dict(anon=True), simplecache={'cache_storage': './tmp'})

ds = xr.open_dataset(tmp_file, engine='cfgrib', 
                       backend_kwargs=dict(filter_by_keys={'typeOfLevel': 'heightAboveGround', 'level': 2}))

In [ ]:
%%time
ds.time.units

In [ ]:
ds = ds.metpy.parse_cf()

In [ ]:
crs = ds.t2m.load(

In [ ]:
ds.t2m

In [ ]:
ds.t2m.hvplot.quadmesh(x='longitude', y='latitude', rasterize=True,
                      crs=crs, tiles='OSM', project=True, alpha=0.5, cmap='viridis')

In [ ]:
ds_uv = xr.open_dataset(tmp_file, engine='cfgrib', 
                        backend_kwargs=dict(filter_by_keys={'typeOfLevel': 'heightAboveGround', 
                                                            'level': 2,
                                                            'stepType': 'instant'}))

In [ ]:
ds_uv

In [ ]:
ds['u10'] = ds_uv.u10
ds['v10'] = ds_uv.v10

In [ ]:
ds.data_vars

In [ ]:
dsn = xr.open_dataset(tmp_file, engine='cfgrib', 
                        backend_kwargs=dict(filter_by_keys={'typeOfLevel': 'surface', 
                                                            'stepType': 'instant'}))

In [ ]:
dsn.data_vars

It's actually faster to convert the gribs to netcdf using wgrib2 and loading those into xarray:
```
# download grib2 files
$ wget https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20170125/conus/hrrr.t07z.wrfsubhf1530.grib2
$ wget https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20170125/conus/hrrr.t08z.wrfsubhf1530.grib2
$ wget https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20170125/conus/hrrr.t09z.wrfsubhf1530.grib2

# convert grib2 to standard netcdf using wgrib2
$ wgrib2 hrrr.t07z.wrfsubhf1530.grib2  -netcdf hrrr.t07z.wrfsubhf1530.nc
$ wgrib2 hrrr.t08z.wrfsubhf1530.grib2  -netcdf hrrr.t08z.wrfsubhf1530.nc
$ wgrib2 hrrr.t09z.wrfsubhf1530.grib2  -netcdf hrrr.t09z.wrfsubhf1530.nc
```

In [ ]:
dsh = xr.open_mfdataset('hrrr*.nc', chunks={'time':72, 'x':600, 'y':600})

In [ ]:
print(dsh.data_vars)

In [ ]:
hfile = 'hrrr.t00z.wrfnatf01.grib2'

In [ ]:
ds = xr.open_dataset(hfile, engine='cfgrib', 
                       backend_kwargs=dict(filter_by_keys={'typeOfLevel': 'heightAboveGround', 'level': 2}))

In [ ]:
ds